## TODO: Frequency and Predictive Power of N-gram Features

Objective: Examine the occurrence of unigrams (single words) and bigrams (two-word combinations) in depression and non-depression posts to identify patterns.

Process:
Compute the frequency of all unigrams and bigrams in both types of posts.
Select the top 100 unigrams and bigrams for each category (depression vs. non-depression).
Analyze high-frequency words and phrases to identify their relevance to depression or standard discussions.

Findings:
Depression-indicative posts:
Focus on self-preoccupation (e.g., "I’m," "I’m not").
Express feelings (e.g., "feel," "feel like").
Negative emotions (e.g., "sad," "depression," "miserable").
Suicidal thoughts (e.g., "want die," "kill").
Signs of hopelessness (e.g., "pointless," "need help").
Physical symptoms (e.g., "tired," "pain," "sleep").
Non-depression posts:
Describe past events (e.g., "month ago," "last year").
Emphasize social relationships (e.g., "friend," "family").
Advice-seeking phrases (e.g., "need advice," "please help").

Predictive Power:
High-frequency depression-indicative words likely improve classification performance since they strongly correlate with depressive language patterns.

## TODO: Predictive Power of Empath Features

Objective: Evaluate the psycholinguistic dimensions of text using the Linguistic Inquiry and Word Count (LIWC) dictionary.

Process:
Map text to 68 psycholinguistic features derived from LIWC categories (e.g., linguistic dimensions, psychological processes, personal concerns).
Convert posts into numerical values based on these features.
Correlate LIWC features with depression-related labels using metrics like Pearson correlation.

Findings:
Highest correlation:
Psychological processes (0.19): Affective processes (e.g., "kill," "cry"), especially negative emotions.
Linguistic dimensions (0.17): Usage of self-references (e.g., "I," "me") and personal pronouns.
Personal concerns (0.16): Discussions of work, money, and death.
Depressed users:
Focus more on self-references, negative emotions, and current/future events.
Discuss sadness, anxiety, and interpersonal processes.

Significance: LIWC features effectively capture psychological and linguistic differences, making them valuable for depression detection models.


## TODO: Predictive Power of LDA Features

Objective: Use Latent Dirichlet Allocation (LDA) to discover hidden topics in the posts, identifying themes associated with depression.

Process:
Train LDA models on the text data.
Generate topic distributions for each post (probability of belonging to each topic).
Select the top 20 topics with the largest proportions in the data.
Use t-SNE for dimensionality reduction to visualize topic clusters in 2D space.

Findings:
Topics indicative of depression include:
Themes like "Depression," "Broke," "Tired," "Pain," reflecting suffering, self-preoccupation, and low self-esteem.
Words associated with disclosure, loneliness, hostility, and interpersonal issues.

Significance: LDA captures latent patterns that are not explicitly defined (unlike LIWC), enabling nuanced analysis of depression-related themes.
